In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import re

from sklearn.decomposition import PCA, SparsePCA,TruncatedSVD
from sklearn import preprocessing
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import silhouette_samples, silhouette_score
from pymongo import MongoClient
import re

from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster


%matplotlib inline

In [114]:
df_c = pd.read_csv('crime_pca_data.csv')

In [4]:
cd Dimension\ Reduction

/Users/evansadler/Desktop/Project Files/Dimension Reduction


In [115]:
df_z = pd.read_csv('zillow_pca_data.csv')

In [116]:
df_f = df_c.merge(df_z, on='hood')

In [117]:
city = 'Seattle'
state = 'WA'
client = MongoClient()
db = client['neighborhood_recommender']
collection = db['hood_data' + '_' + city + '_' + state]

hood_zillow = [hood for hood in collection.find()]
df_zo = pd.DataFrame(hood_zillow, dtype = np.float64)
names = df_zo['name'].values
df_zo = df_zo[['id', 'name']]
df_zo.columns = ['id', 'hood']

In [118]:
df_f = df_f.merge(df_zo, on='hood')

In [29]:
from combine_data import CombineData

In [ ]:
cb = CombineData('Seattle', 'WA')
cb.load_data_and_engineer_features()

In [119]:
df_y = pd.read_csv('yelp_pca_data.csv')

In [120]:
df_y.rename(columns = {'hood_id': 'id'}, inplace = True)

In [121]:

df_f = df_f.merge(df_y, on = 'id')

In [122]:
w = WalkscoreFeatures('Seattle', 'WA')
df_ws = w.engineer_features()
df_ws['id'] = df_ws['id'].astype(int)

In [44]:
from walkscore_features import WalkscoreFeatures

In [124]:
df_f = df_ws.merge(df_f, on ='id')

In [79]:
names = df_f['hood']
df_f.drop('id', axis = 1, inplace = True)

In [65]:
from pca_class import ReduceFeatures

In [88]:
rf = ReduceFeatures(df_f, names)
rf.preprocessing(0)
#rf.hcluster_cols(0.3)
#rf.preprocessing(0)
df_c = rf.sparse_pca(n_components = 5, alpha = 1)
rf.best_cluster()

For n_clusters = 3 The average silhouette_score is : 0.285476657747
For n_clusters = 4 The average silhouette_score is : 0.270739924881
For n_clusters = 5 The average silhouette_score is : 0.286060148861
For n_clusters = 6 The average silhouette_score is : 0.280979887418
For n_clusters = 7 The average silhouette_score is : 0.271517841639
For n_clusters = 8 The average silhouette_score is : 0.259430615592
For n_clusters = 9 The average silhouette_score is : 0.283117971254
For n_clusters = 10 The average silhouette_score is : 0.269825435618


In [92]:
df_c

,1,2,3
transit_score,-6.590399,0.000000,0.000000
walkscore,-6.231634,0.000000,0.283360
crime,5.893753,0.000000,0.000000
serious crime,0.000000,-2.542314,-3.561765
prostitution,-3.870505,-3.160507,0.000000
family vs single,6.271583,0.000000,0.000000
expensiveness,0.000000,6.818006,0.000000
Age,0.000000,0.000000,-6.842315


In [125]:
df_f.to_csv('final_1.csv', index = False)

In [113]:
df_f

,id,transit_score,walkscore,hood,crime,serious crime,prostitution,family vs single,expensiveness,Age,active,arts,nightlife,restaurants,shopping


In [127]:
df_f.ix[df_f['hood'] == 'Beacon Hill', :]

,id,transit_score,walkscore,hood,crime,serious crime,prostitution,family vs single,expensiveness,Age,active,arts,nightlife,restaurants,shopping
1,250050,53.96,61.32,Beacon Hill,-0.080443,0.009634,0.519038,0.113373,-0.115577,0.066851,75,47,32,179,170
